In [7]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import requests

In [8]:
subway = pd.read_csv('./data/지하철위도경도(신).csv')
subway

,외구간_역_수,역한글명칭,호선명칭,환승역X좌표,환승역Y좌표
0,4128,삼성중앙,9호선(연장),127.053282,37.513011
1,4124,사평,9호선,127.015259,37.504206
2,4121,구반포,9호선,126.987332,37.501364
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770
4,4116,샛강,9호선,126.928422,37.517274
...,...,...,...,...,...
760,4411,관악산(서울대),신림선,126.945064,37.469102
761,4306,신논현,신분당선(연장2),127.025060,37.504598
762,1404,탕정,장항선,127.084850,36.788660
763,4304,신사,신분당선(연장2),127.020114,37.516334


In [9]:
# 카카오맵 API Key
kakao_api_key = '146c824df784569973cb17777ddd3301'

subway['시군구'] = None
subway['행정동'] = None

# 카카오맵 API를 호출하여 주소별 위도, 경도 및 행정동명을 얻습니다.
for index, row in subway.iterrows():
    latitude = row['환승역Y좌표']
    longitude = row['환승역X좌표']
    
    url = f'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('documents'):
            administrative_district1 = data['documents'][1]['region_2depth_name']
            administrative_district2 = data['documents'][1]['region_3depth_name']
            subway.at[index, '시군구'] = administrative_district1
            subway.at[index, '행정동'] = administrative_district2

# 결과를 출력합니다.
subway


,외구간_역_수,역한글명칭,호선명칭,환승역X좌표,환승역Y좌표,시군구,행정동
0,4128,삼성중앙,9호선(연장),127.053282,37.513011,강남구,삼성2동
1,4124,사평,9호선,127.015259,37.504206,서초구,반포1동
2,4121,구반포,9호선,126.987332,37.501364,서초구,반포본동
3,4119,흑석(중앙대입구),9호선,126.963708,37.508770,동작구,흑석동
4,4116,샛강,9호선,126.928422,37.517274,영등포구,여의동
...,...,...,...,...,...,...,...
760,4411,관악산(서울대),신림선,126.945064,37.469102,관악구,대학동
761,4306,신논현,신분당선(연장2),127.025060,37.504598,강남구,역삼1동
762,1404,탕정,장항선,127.084850,36.788660,아산시,탕정면
763,4304,신사,신분당선(연장2),127.020114,37.516334,강남구,논현1동


In [10]:
subway.drop(columns=['외구간_역_수', '호선명칭', '환승역X좌표', '환승역Y좌표'], inplace=True)
subway

,역한글명칭,시군구,행정동
0,삼성중앙,강남구,삼성2동
1,사평,서초구,반포1동
2,구반포,서초구,반포본동
3,흑석(중앙대입구),동작구,흑석동
4,샛강,영등포구,여의동
...,...,...,...
760,관악산(서울대),관악구,대학동
761,신논현,강남구,역삼1동
762,탕정,아산시,탕정면
763,신사,강남구,논현1동


In [23]:
subway.to_csv('./data/h지하철역주소.csv')
subway

,역한글명칭,시군구,행정동
0,삼성중앙,강남구,삼성2동
1,사평,서초구,반포1동
2,구반포,서초구,반포본동
3,흑석(중앙대입구),동작구,흑석동
4,샛강,영등포구,여의동
...,...,...,...
760,관악산(서울대),관악구,대학동
761,신논현,강남구,역삼1동
762,탕정,아산시,탕정면
763,신사,강남구,논현1동


In [11]:
subway_dongbuk = subway[(subway['시군구'] == '강북구') | (subway['시군구'] == '도봉구') | (subway['시군구'] == '노원구')| (subway['시군구'] == '중랑구')
                            | (subway['시군구'] == '성북구')| (subway['시군구'] == '동대문구')]
subway_dongbuk

,역한글명칭,시군구,행정동
10,청량리(서울시립대입구),동대문구,전농1동
15,한성대입구(삼선교),성북구,삼선동
20,수유(강북구청),강북구,번1동
26,제기동,동대문구,용신동
47,망우,중랑구,상봉1동
...,...,...,...
667,용두(동대문구청),동대문구,용신동
670,당고개,노원구,상계3.4동
680,도봉,도봉구,도봉2동
681,회기,동대문구,휘경1동


In [12]:
subway_dongbuk.to_csv('./data/h지하철주소동북.csv')
subway_dongbuk

,역한글명칭,시군구,행정동
10,청량리(서울시립대입구),동대문구,전농1동
15,한성대입구(삼선교),성북구,삼선동
20,수유(강북구청),강북구,번1동
26,제기동,동대문구,용신동
47,망우,중랑구,상봉1동
...,...,...,...
667,용두(동대문구청),동대문구,용신동
670,당고개,노원구,상계3.4동
680,도봉,도봉구,도봉2동
681,회기,동대문구,휘경1동


In [24]:
subway_dongbuk[subway_dongbuk['시군구'] == '노원구']

,역한글명칭,시군구,행정동
174,상계,노원구,상계5동
326,공릉(서울과학기술대),노원구,공릉1동
327,하계,노원구,하계2동
328,노원,노원구,상계6.7동
331,태릉입구,노원구,공릉1동
455,광운대,노원구,월계1동
501,노원,노원구,상계2동
548,중계,노원구,중계2.3동
549,수락산,노원구,상계1동
550,화랑대(서울여대입구),노원구,공릉2동


In [13]:
grouped = subway_dongbuk.groupby(['시군구', '행정동']).size().reset_index(name='역수')
grouped

,시군구,행정동,역수
0,강북구,미아동,1
1,강북구,번1동,1
2,강북구,삼양동,3
3,강북구,송중동,1
4,강북구,수유1동,1
5,강북구,수유2동,1
6,강북구,우이동,3
7,노원구,공릉1동,3
8,노원구,공릉2동,1
9,노원구,상계1동,1


In [14]:
grouped.to_csv('./data/동북부지하철수.csv')
grouped

,시군구,행정동,역수
0,강북구,미아동,1
1,강북구,번1동,1
2,강북구,삼양동,3
3,강북구,송중동,1
4,강북구,수유1동,1
5,강북구,수유2동,1
6,강북구,우이동,3
7,노원구,공릉1동,3
8,노원구,공릉2동,1
9,노원구,상계1동,1


In [18]:
width = pd.read_csv('./data/h동북부면적.csv')
width

,시군구,행정동,면적
0,동대문구,회기동,0.76
1,동대문구,휘경1동,0.63
2,동대문구,휘경2동,1.05
3,동대문구,청량리동,1.20
4,동대문구,용신동,1.61
...,...,...,...
91,노원구,상계10동,0.80
92,노원구,상계3.4동,5.05
93,노원구,상계6.7동,1.11
94,노원구,중계2.3동,0.90


In [19]:
width = pd.merge(width, grouped, on=['시군구', '행정동'], how='left')
width

,시군구,행정동,면적,역수
0,동대문구,회기동,0.76,NaN
1,동대문구,휘경1동,0.63,1.0
2,동대문구,휘경2동,1.05,NaN
3,동대문구,청량리동,1.20,NaN
4,동대문구,용신동,1.61,5.0
...,...,...,...,...
91,노원구,상계10동,0.80,NaN
92,노원구,상계3.4동,5.05,1.0
93,노원구,상계6.7동,1.11,1.0
94,노원구,중계2.3동,0.90,1.0


In [14]:
width['역수'].fillna(0, inplace=True)
width

,시군구,법정동,면적,역수
0,동대문구,신설동,0.22,2.0
1,동대문구,용두동,0.76,2.0
2,동대문구,제기동,0.72,0.0
3,동대문구,전농동,0.94,2.0
4,동대문구,답십리동,1.02,0.0
5,동대문구,장안동,1.47,0.0
6,동대문구,청량리동,0.62,0.0
7,동대문구,회기동,0.28,0.0
8,동대문구,휘경동,0.86,1.0
9,동대문구,이문동,0.91,2.0


In [20]:
width['면적(㎢)당 역 수'] = width['역수'] / width['면적']
width

,시군구,행정동,면적,역수,면적(㎢)당 역 수
0,동대문구,회기동,0.76,NaN,NaN
1,동대문구,휘경1동,0.63,1.0,1.587302
2,동대문구,휘경2동,1.05,NaN,NaN
3,동대문구,청량리동,1.20,NaN,NaN
4,동대문구,용신동,1.61,5.0,3.105590
...,...,...,...,...,...
91,노원구,상계10동,0.80,NaN,NaN
92,노원구,상계3.4동,5.05,1.0,0.198020
93,노원구,상계6.7동,1.11,1.0,0.900901
94,노원구,중계2.3동,0.90,1.0,1.111111


In [21]:
width.drop(columns=['면적', '역수'], inplace=True)
width

,시군구,행정동,면적(㎢)당 역 수
0,동대문구,회기동,NaN
1,동대문구,휘경1동,1.587302
2,동대문구,휘경2동,NaN
3,동대문구,청량리동,NaN
4,동대문구,용신동,3.105590
...,...,...,...
91,노원구,상계10동,NaN
92,노원구,상계3.4동,0.198020
93,노원구,상계6.7동,0.900901
94,노원구,중계2.3동,1.111111


In [22]:
width.to_csv('./data/h동북부지하철수.csv')
width

,시군구,행정동,면적(㎢)당 역 수
0,동대문구,회기동,NaN
1,동대문구,휘경1동,1.587302
2,동대문구,휘경2동,NaN
3,동대문구,청량리동,NaN
4,동대문구,용신동,3.105590
...,...,...,...
91,노원구,상계10동,NaN
92,노원구,상계3.4동,0.198020
93,노원구,상계6.7동,0.900901
94,노원구,중계2.3동,1.111111
